# Reactive Power Optimization with PandaModels.jl

This tutorial describes how to solve some optimization problems regarding reactive power redispatch using **PandaModels**. The reactive power provision of generators, e.g., net.sgen, can be optimized for  

1) maintaining voltage setpoints  
2) maintaining reactive power setpoints  
3) (active) power loss reduction \
4) branch loading reduction


For more details on the corresponding models, see:
* [maintaining voltage setpoints](https://github.com/e2nIEE/PandaModels.jl/blob/develop/src/models/vstab.jl)
* [maintaining reactive power setpoints](https://github.com/e2nIEE/PandaModels.jl/blob/develop/src/models/qflex.jl)
* [power loss reduction](https://github.com/e2nIEE/PandaModels.jl/blob/develop/src/models/ploss.jl)
* [branch loading reduction](https://github.com/e2nIEE/PandaModels.jl/blob/develop/src/models/loading.jl)

**Note**:

- **PandaModels** is still in the development stage, and the developed models for reactive power optimizations are tested with limited grids. It is possible that errors may arise when facing different grids.
- It is suggested to use **net.sn_mva=1.0** to start the optimization. Using other sn_mva values may introduce not converged OPF  


## 1) Maintaining Voltage Setpoints

Grid operators want to maintain a constant voltage profile across all or a part of the system buses. Reactive power optimization, in this case, helps them identify the preferable provision of reactive power from generators. The objective is to minimize the deviation of the voltages of certain buses (here, buses of DER) from a setpoint, e.g.,$V_{\rm setpoint}
= 1.03 p.u.$ As a result, the reactive power setpoint for each generator is calculated. The mathematical formulation of this objective function can have the following forms:

\begin{align}
& \underset{\mathcal{X} = [q, ...]}{\text{minimize}}
& &  \sum_{i\in \mathcal{BI}} (V_i-V_{\rm setpoint})^2 \\
& \text{subject to}
& & g(\mathcal{X})=0 \\
& & & h(\mathcal{X}) \leq 0
\end{align}

where $V_{i}$ is the voltage variable of bus $i$ in $\mathcal{BI}$ which denotes the set of buses of DER. The $g(\mathcal{X})$ and $h(\mathcal{X})$, denote equality and inequality constraints, respectively. The $\mathcal{X}$ denotes the set of variables decisions, such as reactive power $q$.

### 1-1: Create test grid

How does it work? Let's first create a cigre grid with DERs(distribution energy resource) from the pandapower's network database. After running the power flow calculation, we can see that there are nine DERs (net.sgen) in the grid, and they don't provide reactive power for normal operation.

In [ ]:
from copy import deepcopy
from pandapower.networks.cigre_networks import create_cigre_network_mv
from pandapower.run import runpp
from pandapower.runpm import runpm_vstab, runpm_multi_vstab, runpm_qflex, runpm_ploss, runpm_loading

In [ ]:
net = create_cigre_network_mv(with_der="pv_wind")
runpp(net) # run power flow calculation
net_org = deepcopy(net) # copy of the grid for further comparison
display(net.res_sgen) # show power flow results for DER

(You can use the pandapower plotting module to visualize the grid model.)

In [ ]:
import pandapower.plotting as plot
plot.simple_plot(net)

### 1-2: Parametrization for optimization
Before starting the optimization, we need to configure some parameters. 
In PandaModels, the uncontrollable loads and DER (net.sgen) are converted to load, and the controllable power elements are converted to generators. Accordingly, we neet to set loads as uncontrollable and set DER (net.sgen) as controllable elements. 

In [ ]:
net_opt = deepcopy(net) # copy of the grid for further comparison 
net_opt.load['controllable'] = False
net_opt.sgen['controllable'] = True

Now, let's set the constraints.

In [ ]:
# lower and upper bounds for buses
net_opt.bus["max_vm_pu"] = 1.1
net_opt.bus["min_vm_pu"] = 0.9

# lower and upper bounds for external grid
net_opt.ext_grid["max_q_mvar"] = 10000.0
net_opt.ext_grid["min_q_mvar"] = -10000.0
net_opt.ext_grid["max_p_mw"] = 10000.0
net_opt.ext_grid["min_p_mw"] = -10000.0

We keep the active power for DER constant to the calculated value from the power flow and define a constraint for the reactive power. (In this grid, there is no generator (net.gen))

In [ ]:
# lower and upper bounds for DERs
net_opt.sgen["max_p_mw"] = net_opt.sgen.p_mw.values
net_opt.sgen["min_p_mw"] = net_opt.sgen.p_mw.values
net_opt.sgen["max_q_mvar"] = net_opt.sgen.p_mw.values * 0.328
net_opt.sgen["min_q_mvar"] = -net_opt.sgen.p_mw.values * 0.328
display(net_opt.sgen)

Subsequently, let's set a high upper bound for lines and transformers to avoid congestion issues.

In [ ]:
# lower and upper bounds for lines
net_opt.trafo["max_loading_percent"] = 100.0
net_opt.line["max_loading_percent"] = 100.0

Finally, we need to let PandaModels know which buses are of interest and what is user-defined voltage setpoint. To do this, we can add extra column called **"pm_param/setpoint_v"** in net.bus and set values for buses connecting DER (net.sgen).

In [ ]:
net_opt.bus["pm_param/setpoint_v"] = None # add extra column
net_opt.bus.loc[net_opt.sgen.bus, "pm_param/setpoint_v"] = 0.96 # voltage setpoints for theses buses are 0.96

### 1-3: Start optimization
Now let's call the function ***pandapower.runpm_vstab()*** to start the optimizaiton for maintaining voltage setpoints. 
(Note that the first time the function is called, Julia is started in the background, which may take some time.)

In [ ]:
try:
    runpm_vstab(net_opt)
except Exception as err:
    print(err)    

Also, the following parameters and options can be modified as inputs while calling the optimization from PandaModles. For this case, we can keep the default values.

| parameter | description | type | default |
| :--- | :--- | :---: | :--- |
| correct_pm_network_data | checks if network data is correct. If not tries to correct it | bool | True |
| silence | Suppresses information and warning messages output by PowerModels | bool | True |
| pm_model | PowerModels.jl model to use | str | "ACPPowerModel" |
| pm_solver | "main" solver| str | "ipopt" |
| pm_mip_solver | mixed integer solver| str | "cbc" |
| pm_nl_solver | nonlinear solver| str | "ipopt" |
| pm_tol | default desired convergence tolerance for solver to use | float | 1e-8 |
| pm_log_level | solver log level in power models | int | 0 |
| delete_buffer_file | If True, the .json file used by PandaModels will be deleted after optimization. | bool | True |



### 1-4: Results comparison
Let's check the results for voltage and reactive power at buses connecting DER before and after the optimization.

In [ ]:
# results from power flow
display(net_org.res_bus.vm_pu[net_org.sgen.bus])
v_org = net_org.res_bus.vm_pu[net_org.sgen.bus].values
print("--- deviation from setpionts v=0.96 ---")
display(sum((v_org-0.96)**2)) 

In [ ]:
# results from optimization
display(net_opt.res_bus.vm_pu[net.sgen.bus])
v_opt = net_opt.res_bus.vm_pu[net_opt.sgen.bus].values
print("--- deviation from setpionts v=0.96 ---")
display(sum((v_opt-0.96)**2)) # devition from setpoints v=0.96 p.u.

As exected, the bus voltages after optimization are close to the defined setpoints. It is caused by the provision of reactive power of DER.

In [ ]:
# results from power flow
display(net_org.res_sgen)

In [ ]:
# results from optimization
display(net_opt.res_sgen)

### 1-5: Saving reactive power with multi-objectives
In some situations, users want to improve the voltage profiles with limited reactive power provisions. For this, the objective function is modified by minimizing the reactive power provision. As the formulation below shows, the two sub-objectives for **maintaining voltage setpoints** and **saving reactive power provisions** are connected through coefficients $c_{\rm v}$ and $c_{\rm q}$. 

\begin{align}
& \underset{\mathcal{X} = [q, ...]}{\text{minimize}}
& &  \sum_{i\in \mathcal{BI}} c_{\rm v}*(V_i-V_{\rm setpoint})^2 + c_{\rm q}*(q_i-0)^2 \\
\end{align}

We can call the objective function with different coefficient combinations, e.g., $c_{\rm v}=0.99$ and $c_{\rm q}=0.01$.
(In default, $c_{\rm v}=1.0$ and $c_{\rm q}=0.0$)

In [ ]:
net_optlim = deepcopy(net_opt)
net_optlim["obj_factors"] = [0.99, 0.01] # set coefficients for the multi-objective function
try:
    runpm_vstab(net_optlim)
except:
    print("[WinError 3] The system cannot find the path specified to the python39.dll")

As the results below show, with $c_{\rm v}=0.99$ and $c_{\rm q}=0.01$, the provided reactive powers are reduced, and the corresponding voltages are increased.

In [ ]:
print ("--- optimized reactive power without limitation Cv = 1.0, Cq = 0. ----")
display(net_opt.res_sgen) # show optimized reactive power without limitation
print ("--- optimized reactive power with limitation Cv = 0.99, Cq = 0.01 ----")
display(net_optlim.res_sgen) # show optimized reactive power with limitation

print ("--- deviation from setpoints without limitation Cv = 1.0, Cq = 0. ----")
display(sum((v_opt - 0.96) ** 2)) # show optimized reactive power without limitation
print ("--- deviation from setpoints with limitation Cv = 0.99, Cq = 0.01 ----")
v_optlim = net_optlim.res_bus.vm_pu[net_optlim.sgen.bus].values
display(sum((v_optlim - 0.96) ** 2)) # show optimized bus voltages

The coefficient combination is grid-dependent. For example, the picture below shows the result for one of our case studies with a realistic grid. The user can find the most suitable combination setting through several attempts.
<img src="pics/coef_combination.png" alt="ALT" width="600">

### 1-6: Time series optimizations
There are two ways to perform time series optimizations. 
The first one is optimization in a loop, i.e., updating the grid condition for each time step and repeatedly calling optimization. The second one uses the "multi-timestep" optimization. PowerModels can solve the multi-grid optimization problem. This feature is used in PandaModels to deal with time series optimization for on specific grid by replacing the "multi-grid" with "multi-timestep" of the grid. 

Next, you will see an example.

First, let's load the time series data.

In [ ]:
import pandas as pd

# load time series data for 96 time steps
time_series = pd.read_json("cigre_timeseries_15min.json")
time_series.sort_index(inplace=True)

# show time series
display(time_series) 

# set load type and sgen type in the cigre grid
net.load.loc[:, "type"] = "residential"
net.sgen.loc[:, "type"] = "pv"
net.sgen.loc[8, "type"] = "wind"

# get rated power
load_p = net.load.loc[:, "p_mw"].values
sgen_p = net.sgen.loc[:7, "p_mw"].values
wind_p = net.sgen.loc[8, "p_mw"]

# create dataframe for loads and sgens
load_ts = pd.DataFrame(index=time_series.index.tolist(), columns=net.load.index.tolist())
sgen_ts = pd.DataFrame(index=time_series.index.tolist(), columns=net.sgen.index.tolist())
for t in range(96):
    load_ts.loc[t] = load_p * time_series.at[t, "residential"]
    sgen_ts.iloc[t, 0:8] = sgen_p * time_series.at[t, "pv"]
    sgen_ts.loc[t, 8] = wind_p * time_series.at[t, "wind"]

Now, we perform the time series optimizations in a loop.

In [ ]:
########################################################################
####################### Optimization in a loop #########################
########################################################################

y_loop = []
for t in range(96):
    
    # Update operating points for each time step
    net_opt.load.p_mw = load_ts.loc[t]
    net_opt.sgen.p_mw = sgen_ts.loc[t]
    
    # If required, we also need to update the constraints, e.g., Qmax- and Qmin-value, for each time step. 
    net_opt.sgen.max_p_mw = sgen_ts.loc[t]   # for load: max_p_mw == min_p_mw, because they are not controllable.
    net_opt.sgen.min_p_mw = sgen_ts.loc[t]
    
    # run optimization for time step t
    try:
        runpm_vstab(net_opt)
    except:
        print("[WinError 3] The system cannot find the path specified to the python39.dll")
    
    # save the mean voltage value for each time step
    y_loop.append(net_opt.res_bus.vm_pu[net_opt.sgen.bus].values.mean())

Alternatively, we can do it using multi-timestep optimization.

In [ ]:
########################################################################
####################### multi-timestep optimization ####################
########################################################################

from pandapower.control import ConstControl
from pandapower.timeseries import DFData

# For this method, we need to add the time series data into the pandapower.controller.
net_opt.controller.drop(net.controller.index, inplace=True) # remove the existing controllers

# Add time series for load and sgen.
ConstControl(net_opt, element="load", variable="p_mw",
             element_index=net.load.index.tolist(), profile_name=net.load.index.tolist(),
             data_source=DFData(load_ts))
ConstControl(net_opt, element="sgen", variable="p_mw",
             element_index=net.sgen.index.tolist(), profile_name=net.sgen.index.tolist(),
             data_source=DFData(sgen_ts))

# If required, we also need to creat a time series for the constraints, e.g., Qmax- and Qmin-value, for each time step. 
df_qmax, df_qmin = sgen_ts.copy(), sgen_ts.copy()
df_qmax[df_qmax.columns] = net_opt.sgen.max_q_mvar
df_qmin[df_qmin.columns] = net_opt.sgen.min_q_mvar

# Then create controllers for them.
ConstControl(net_opt, element="sgen", variable="max_p_mw",
              element_index=net.sgen.index.tolist(), profile_name=net.sgen.index.tolist(),
              data_source=DFData(sgen_ts))
ConstControl(net_opt, element="sgen", variable="min_p_mw",
              element_index=net.sgen.index.tolist(), profile_name=net.sgen.index.tolist(),
              data_source=DFData(sgen_ts))
ConstControl(net_opt, element="sgen", variable="max_q_mvar",
              element_index=net.sgen.index.tolist(), profile_name=net.sgen.index.tolist(),
              data_source=DFData(df_qmax))
ConstControl(net_opt, element="sgen", variable="min_q_mvar",
              element_index=net.sgen.index.tolist(), profile_name=net.sgen.index.tolist(),
              data_source=DFData(df_qmin))

# For the multi-timestep optimization, we need to call another PandaModels-function.
try:
    runpm_multi_vstab(net_opt, from_time_step=0, to_time_step=96)
except:
    print("[WinError 3] The system cannot find the path specified to the python39.dll")

**Note that**, currently, the function ***runpm_multi_vstab()*** doesn't support using of the multi-objective function for saving reactive power.

The results for each time step are saved in **net.res_ts_opt**. Let's get them.

In [ ]:
y_multi = []
for t in range(96):
    y_multi.append(net_opt.res_ts_opt[str(t)].res_bus.vm_pu[net_opt.sgen.bus].values.mean())

And then plot them.

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True)
ax1.plot(range(96), y_loop)
ax1.set_title("results for \n optimization in a loop")
ax2.plot(range(96), y_multi)
ax2.set_title("results for \n multi-timestep optimization")

for ax in ax1, ax2:
    ax.grid(True)
    ax.label_outer()
    ax.set_xlabel("time step [15min]")
ax1.set_ylabel("mean voltage value for DER-buses [p.u.]")

As the plot above shows, both methods have the same results.

**For a long-term time series optimization (>500 timesteps), we suggest using the first method, optimization in a loop.**

## 2) Maintaining Reactive Power Setpoints

The motivation for this objective is to minimize the deviations of reactive power injection at the TSO-DSO interface from a target value, e.g., $Q_{\rm setpoint}= 2 \rm MVar$. It is formulated by the following equation:

\begin{align}
& \underset{\mathcal{X} = [q, ...]}{\text{minimize}}
& &  \sum_{i\in \mathcal{interfaces}} (Q_i-Q_{\rm setpoint})^2 \\
\end{align}

We use the same grid model to introduce the example. As the picture low shows, the marked bus (or interface) is observed.


<img src="pics/qflex.png" alt="ALT" width="300">

First, let's create the grid and check the reactive power injection at this interface.

In [ ]:
net = create_cigre_network_mv(with_der="pv_wind")
runpp(net)
net_org = deepcopy(net)

display(net.res_trafo.q_lv_mvar[0])

Without optimization, the reactive power injection at this interface is -5.942 MVAr.

Let's configure optimization parameters and define a target value for this interface, $Q_{\rm setpoint}= -5 \rm MVAr$, i.e., we optimize the reactive power provision of DER to achieve this target.

In [ ]:
net_opt = deepcopy(net) # copy of the grid for further comparison 
net_opt.load['controllable'] = False
net_opt.sgen['controllable'] = True

# lower and upper bounds for buses
net_opt.bus["max_vm_pu"] = 1.1
net_opt.bus["min_vm_pu"] = 0.9

# lower and upper bounds for external grid
net_opt.ext_grid["max_q_mvar"] = 10000.0
net_opt.ext_grid["min_q_mvar"] = -10000.0
net_opt.ext_grid["max_p_mw"] = 10000.0
net_opt.ext_grid["min_p_mw"] = -10000.0

# lower and upper bounds for DERs
net_opt.sgen["max_p_mw"] = net_opt.sgen.p_mw.values
net_opt.sgen["min_p_mw"] = net_opt.sgen.p_mw.values
net_opt.sgen["max_q_mvar"] = net_opt.sgen.p_mw.values * 0.328
net_opt.sgen["min_q_mvar"] = -net_opt.sgen.p_mw.values * 0.328
display(net_opt.sgen)

# lower and upper bounds for lines
net_opt.trafo["max_loading_percent"] = 100.0
net_opt.line["max_loading_percent"] = 100.0

######################################################
####################### Attention ####################
######################################################

# We need to let PandaModels know which interface is of interest and what is user-defined setpoint. 
# So, we need to add extra column called **"pm_param/setpoint_q"** in net.trafo and set target value. 

net_opt.trafo["pm_param/setpoint_q"] = None # add extra column
net_opt.trafo["pm_param/setpoint_q"][0] = -5 #  reactive power setpoint for trafo 0 is -5 MVAR

# Also, we need to tell PandaModels which side is observed.
net_opt.trafo["pm_param/side"] = None
net_opt.trafo["pm_param/side"][0] = "lv" # the observed side is the low-voltage side


After parametrization, we can start the optimization using ***runpm_qflex()***.

In [ ]:
try:
    runpm_qflex(net_opt)
except:
    print("[WinError 3] The system cannot find the path specified to the python39.dll")

Let's check the optimization results.

In [ ]:
display(net_opt.res_trafo.q_lv_mvar[0])  # show Q-injection at the interface after optimization
display(net_opt.res_sgen) # show optimized reactiv power provision of DER

From the results above, we can see that the Q-injection at the interface is closer to the setpoint after the optimization but not achieved. The reason is that the Q-maximum is limited. We can modify the Q-maximum and try again.

In [ ]:
net_opt.sgen["max_q_mvar"] = net_opt.sgen.p_mw.values * 0.628
net_opt.sgen["min_q_mvar"] = -net_opt.sgen.p_mw.values * 0.628
try:
    runpm_qflex(net_opt)
except:
    print("[WinError 3] The system cannot find the path specified to the python39.dll")
display(net_opt.res_trafo.q_lv_mvar[0])  # show Q-injection at the interface after optimization
display(net_opt.res_sgen)

Now, the Q-injection at the interface is optimized to -5 MVAr. 

Similar to the **maintaining voltage setpoints**, if you are interested in time series optimizations, you can do it through a loop or use ***runpm_multi_qflex()***.

## 3) Power Loss Reduction and Branch Loading Reduction

The corresponding objective functions are formulated as follows: 

- loss reduction
\begin{align}
& \underset{\mathcal{X} = [q, ...]}{\text{minimize}}
& &  \sum_{(i, j)\in \mathcal{N}} (S_{ij}+S_{ji})^2 \\
\end{align}


- loading reduction
\begin{align}
& \underset{\mathcal{X} = [q, ...]}{\text{minimize}}
& &  \sum_{(i, j)\in \mathcal{N}} (i_{i->j, target}-0)^2 \\
\end{align}

We still use the cigre medium voltage grid to introduce the case studies. First, let's create the grid, add constraints, and define the observed branches.

In [ ]:
net = create_cigre_network_mv(with_der="pv_wind")
net.load['controllable'] = False
net.sgen['controllable'] = True
net.sgen["max_p_mw"] = net.sgen.p_mw.values
net.sgen["min_p_mw"] = net.sgen.p_mw.values
net.sgen["max_q_mvar"] = net.sgen.p_mw.values * 0.328
net.sgen["min_q_mvar"] = -net.sgen.p_mw.values * 0.328
net.bus["max_vm_pu"] = 1.1
net.bus["min_vm_pu"] = 0.9
net.ext_grid["max_q_mvar"] = 10000.0
net.ext_grid["min_q_mvar"] = -10000.0
net.ext_grid["max_p_mw"] = 10000.0
net.ext_grid["min_p_mw"] = -10000.0
net.trafo["max_loading_percent"] = 100.0
net.line["max_loading_percent"] = 100.0

# add an additional column named "pm_param/target_branch" to let PandaModels know the observed branches.
net.line["pm_param/target_branch"] = True  
net.switch.loc[:, "closed"] = True

# run power flow calculation
runpp(net)
net_loss = deepcopy(net)
net_loading = deepcopy(net)

As you see, the power flow results without optimization are stored in the grid ***net***, and the grids ***net_loss*** and ***net_loading*** are going to used for optimization. Now we call the functions ***runpm_ploss*** and ***runpm_loading*** to reduce the power loss and line loading percent.

In [ ]:
runpm_ploss(net_loss)
runpm_loading(net_loading)

Let's compare the results.

In [ ]:
print("loss without optimization:", net.res_line.pl_mw.values.sum())
print("loss with optimization:", net_loss.res_line.pl_mw.values.sum())
print("++++++++++++++++++++++++++++++++++++++++++++++")
print("loading without optimization:", net.res_line.loading_percent.values.sum())
print("loading with optimization:", net_loading.res_line.loading_percent.values.sum())

Also, we can reduce loss or loading percent of transformers:

In [ ]:
net_loss.line.drop(columns=["pm_param/target_branch"], inplace=True)
net_loss.trafo["pm_param/target_branch"] = True
runpm_ploss(net_loss)
print("loss without optimization:", net.res_trafo.pl_mw.values.sum())
print("loss with optimization:", net_loss.res_trafo.pl_mw.values.sum())